In [3]:
import cv2
import numpy as np
import torch
from mtcnn.mtcnn import MTCNN
import pickle
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image, ImageEnhance

In [4]:
torch.cuda.is_available()

False

In [10]:
# Tải mô hình FaceNet đã được huấn luyện
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
facenet_model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

# Tải cơ sở dữ liệu khuôn mặt
with open('face_database.pkl', 'rb') as f:
    database = pickle.load(f)

# Hàm tăng cường dữ liệu và chuẩn hóa hình ảnh
def preprocess_image(image):
    # Tăng độ sáng và độ tương phản của hình ảnh
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(1.5)
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.5)
    
    # Áp dụng các phép biến đổi ảnh
    transform = transforms.Compose([
        transforms.Resize((160, 160)),
        transforms.ColorJitter(brightness=0.5, contrast=0.5),
        transforms.RandomHorizontalFlip(),  # Lật ngang ngẫu nhiên
        transforms.RandomRotation(10),  # Xoay ngẫu nhiên
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
    return transform(image).unsqueeze(0).to(device)

# Hàm chuyển đổi hình ảnh sang embedding sử dụng FaceNet
def img_to_encoding(img_input, model):
    try:
        img = Image.fromarray(img_input).convert('RGB')
        img_tensor = preprocess_image(img)
        with torch.no_grad():
            encoding = model(img_tensor).cpu().numpy()
        return encoding
    except Exception as e:
        print(f"Error processing image: {e}")
        return None

# Hàm nhận diện khuôn mặt sử dụng embeddings của FaceNet
def recognize_face(face_image, database, model):
    encoding = img_to_encoding(face_image, model)
    if encoding is None:
        return 'Unknown', 1

    max_accuracy = 1
    identity = None

    for name, db_encoding in database.items():
        accuracy = np.linalg.norm(db_encoding - encoding)

        print(f'Accuracy for {name} is {1 - accuracy}')

        if accuracy < max_accuracy:
            max_accuracy = accuracy
            identity = name

    if identity is not None and max_accuracy < 0.5:  # Điều chỉnh ngưỡng khi cần thiết
        return identity, max_accuracy
    else:
        return 'Unknown', 1

# Hàm trích xuất thông tin khuôn mặt và nhận diện khuôn mặt trong luồng video
def extract_face_info(img, img_rgb, database, model):
    detector = MTCNN()
    faces = detector.detect_faces(img_rgb)

    if faces:
        for face in faces:
            x, y, w, h = face['box']
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 0), 2)
            face_image = img[y:y + h, x:x + w]
            face_image_rgb = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
            name_ID, accuracy = recognize_face(face_image_rgb, database, model)
            if accuracy < 0.5:  # Điều chỉnh ngưỡng khi cần thiết
                cv2.putText(img, "Name : " + name_ID , (x, y - 50), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
                cv2.putText(img, "Accuracy : " + str(round((1-accuracy)*100,2)) + "%", (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)
            else:
                cv2.putText(img, 'Unknown', (x, y - 20), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 255), 2)

# Hàm nhận diện khuôn mặt trong luồng video trực tiếp sử dụng FaceNet
def recognize():
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FPS, 60)  

    while True:
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        # Trích xuất thông tin khuôn mặt và nhận diện khuôn mặt sử dụng MTCNN và FaceNet
        extract_face_info(img, img_rgb, database, facenet_model)
        
        cv2.imshow('Recognizing faces', img)
        if cv2.waitKey(1) == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Gọi hàm recognize để bắt đầu nhận diện khuôn mặt
recognize()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 128ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Accuracy for Chau Dang Khoa_1 is 0.5581535398960114
Accuracy for Khoa Nguyen_3 is 0.10773152112960815
Accuracy for Phan Phu Tuan_2 is -0.04339635372161865
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1